## Two planets

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from array import *
import random
from scipy import *
from scipy.spatial.distance import pdist
import numpy as np
from matplotlib import cm, colors, pyplot as plt
from matplotlib import animation
from IPython.display import HTML
import copy
import ffmpeg
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import sys
%matplotlib inline

# Change to PLanet600.dat to get head-on collision used in the report results
# The 1200 is for the massive collision
statevec = np.loadtxt('Planet300.dat')
statevec2 = np.loadtxt('Planet300.dat')
statevec[:, 0] = statevec[:, 0] + 1e8
statevec[:, 3] = statevec[:, 3] - 5e4

# More massive collision
#statevec[:, 6] = statevec[:, 6]*1.5

# Skewed collision
statevec[:, 2] = statevec[:, 2] + 3.5e7

statevec2[:, 0] = statevec2[:, 0] - 1e8
statevec2[:, 3] = statevec2[:, 3] + 5e4

inn = np.append(statevec, statevec2, axis = 0)

gamma = 1.4

plt.style.use('presentation-Copy1.mplstyle')

# Properties for the integrator
time_step   = 0.1
steps       = 1 # Number of steps to integrate

# Global parameters
N = len(inn)
G = 6.67408e-11


def Smooth(x,h):
    '''The Kernel function or smoothing function used from the lecture notes'''
    kappa = 2
    H = (h + h[:,np.newaxis])/2
    alpha = 3/(2*np.pi*H**3)  # Three dimensional
    sl = kappa*H
    dx = x - x[:,np.newaxis]
    r = np.linalg.norm(dx, axis = 2)

    R = r/H
 
    # Conditions for kernel function
    R1 = (r < H) & (R >= 0) # since R = |dx|/h < 1 --> |dx| < h
    R2 = (H <= r) & (r < 2*H) # Since R = |dx|/h < 2 --> |dx| < 2*h 
    
    W = np.zeros((N,N))
    
    # solve equation with conditions  

    W[R1] = (2/3) - (R[R1]**2) + (1/2)*(R[R1]**3)
    W[R2] = (1/6)*(2- R[R2])**3
    val_W = alpha*W
    
    # Derivative

    R = R.reshape(N,N,1)
    r = r.reshape(N,N,1)  
    H = H.reshape(N,N,1)
    alpha = alpha.reshape(N,N,1)
    
    
    dW = np.zeros((N,N,3))
    dW[R1] = (-2 + (3/2)*R[R1]) * (dx[R1])/(H[R1]**2)
    dW[R2] = - (1/2)*((2 - R[R2])**2) * (dx[R2])/(H[R2]*r[R2])
    dW[R2] = np.nan_to_num(dW[R2])
 
    val_dW = alpha*dW

    return val_W, val_dW, H

def gravpotder(x, h):
    '''The force kernel'''
    H = (h + h[:,np.newaxis])/2
    dx = x - x[:,np.newaxis]
    r = np.linalg.norm(dx, axis = 2)
    
    R = r/H
    
    # Introducing the unit vector, pointing where particles feel resulting gravity
    unitv = dx/(r.reshape(N,N,1))
    unitv = np.nan_to_num(unitv)
    
    dF = np.zeros((N,N))

    # Conditions
    dC1 = (0 <= R) & (R <= 1)
    dC2 = (1 <= R) & (R <= 2)
    dC3 = (R >= 2)
    
    dF[dC1] = (1/(H[dC1]**2))*((4/3)*R[dC1] - (6/5)*R[dC1]**3 + 0.5*R[dC1]**4)
    dF[dC2] = (1/(H[dC2]**2))*((8/3)*R[dC2] - 3*R[dC2]**2 + (6/5)*R[dC2]**3 - (1/6)*(R[dC2]**4) - (1/(15*R[dC2]**2)))
    dF[dC3] = 1/(r[dC3]**2)
               
    return unitv*(dF.reshape(N,N,1))

def gravpot(x, h):
    '''The gravitaional potential from lecture notes'''
    H = (h + h[:,np.newaxis])/2
    dx = x - x[:,np.newaxis]
    r = np.linalg.norm(dx, axis = 2)
    
    R = r/H
    
    F = np.zeros((N,N))
    
    C1 = (r <= H) & (R >= 0)
    C2 = (H <= r) & (r <= 2*H)
    C3 = (R >= 2)

    F[C1] = (1/H[C1]) * ((2/3)*R[C1]**2 - (10/3)*R[C1]**3 + (1/10)*R[C1]**5 - (7/5))
    F[C2] = (1/H[C2]) * ((4/3)*R[C2]**2 - R[C2]**3 + (3/10)*R[C2]**4 - (1/30)*R[C2]**5 - (8/5) + (1)/(15*R[C2]))
    F[C3] = -1/r[C3]
        
    return F

    
    
def Derivatives(t, r, boundary = False):
    '''This function calculates the evolution of the properties: Density, pressure, position, internal energy
    and velocity for the particles. The state vector r is evolved using an inbuilt integrator the solves
    the coupled ODEs, given as the co-moving derivatives in the Navier stokes equations.
    If you want to include no-slip boundary condition, set boundary = True'''
    
    # Constants
    gamma = 1.4
    eta = 1.2

    # What is to be solved: State vector
    x = np.array(r[:N])
    y = np.array(r[N:2*N])
    z = np.array(r[2*N:3*N])
    P = np.array(r[3*N:4*N])
    rho = np.array(r[4*N:5*N])
    v = np.array(r[5*N:6*N])
    vy = np.array(r[6*N:7*N])
    vz = np.array(r[7*N:8*N])
    e = np.array(r[8*N:9*N])
    m = r[9*N:10*N]
    c = np.array(r[10*N:11*N])
    
    # Divide position and velocity into components 
    posi = np.zeros((N,3))
    posi[:, 0] = x
    posi[:, 1] = y
    posi[:, 2] = z
    
    veli = np.zeros((N,3))
    veli[:, 0] = v
    veli[:, 1] = vy
    veli[:, 2] = vz

    # Smoothing length
    h = np.ones((len(rho),))*1e7 # Use for more stable solution
    #h = eta*(m/rho)**(1/3)
    hij = (1/2)*(h + h[:, np.newaxis])
    hij = hij.reshape(N,N)
    
    # Kernel function values
    W, dW, DX = Smooth(posi, h) # dx = velocity derivative
    
    #Force kernel values
    F = gravpot(posi, h)
    dF = gravpotder(posi, h)
    
    # Density
    sum_dens = m[:, np.newaxis]*W
    rho = np.sum(sum_dens, axis = 1)
    
    # Pressure 
    P = (gamma-1)*rho*e

    # Update
    r[3*N:4*N] = P
    r[4*N:5*N] = rho
   
    # Viscosity equations
    phiij = 0.1*hij
    phiij = phiij.reshape(N,N)
    
    # Speed of sound
    c = np.sqrt((gamma-1)*e)
    c = np.nan_to_num(c)
    cij= 0.5*(c + c[:,np.newaxis])
    
    # Distance and velocity
    vij = veli - veli[:, np.newaxis]
    xij = posi - posi[:, np.newaxis]

    vxij = np.sum(vij*xij, axis = 2)
    xxij = np.sum(xij**2, axis = 2)
    
    # Change of density
    rhoij = 0.5*(rho + rho[:, np.newaxis])
    cond = vxij
    
    # Conditions for largepi 
    cond1 = (cond < 0)
    cond2 = (cond >= 0)
    PIij = np.zeros(cond1.shape)

    varphiij = (hij*vxij)/((xxij) + phiij**2)
    
    PIij[cond1] = (-cij[cond1]*varphiij[cond1] + varphiij[cond1]**2)/(rhoij[cond1])
    PIij[cond2] = 0

    largepi = PIij

    # i and j values of pressure and density
    PI = P
    PJ = P[:, np.newaxis]
    rhoi = rho
    rhoj = rho[:, np.newaxis]
    pressure_and_density_fraction = ((PI/(rhoi**2)) + (PJ/(rhoj**2)) + largepi)*m[:, np.newaxis]
    
    
    # Acceleration gravity
    Gi = gravpotder(posi, h)
    Gj = gravpotder(posi, h.T)
    gdvdt = - (G/2)*np.sum(m[:, np.newaxis]*(Gi + Gj), axis = 0)

    dvx_calc = pressure_and_density_fraction*dW[:, :, 0]
    dvx = -np.sum(dvx_calc, axis=0) + gdvdt[:, 0]
    
    dvy_calc = pressure_and_density_fraction*dW[:, :, 1]
    dvy = -np.sum(dvy_calc, axis=0) + gdvdt[:, 1]
    
    dvz_calc = pressure_and_density_fraction*dW[:, :, 2]
    dvz = -np.sum(dvz_calc, axis=0) + gdvdt[:, 2]
    
    #print(dvx)
    #sys.exit()
    
    # Dirichlet no-slip boundary condition in shock tube
    if boundary:
        Z1 = (-0.6*0.95 >= x)                     
        Z2 = (x >= 0.6*0.95)                    

        dv[Z1] = 0            
        dv[Z2] = 0
    
    # Energy 
    ss = (m[:, np.newaxis])*(PI/(rhoi**2) + PJ/(rhoj**2) + largepi)
    
    de = np.einsum('ij,ijk->ijk', ss, vij)
    de = np.einsum('ijk, ijk->ij', de, dW)
    de = np.einsum('ij->i', de)
    de = 0.5*de

    # velocity
    dxx = veli[:, 0]
    dyy = veli[:, 1]
    dzz = veli[:, 2]
    
    # Derivatives 
    drho = np.zeros(N)
    dP = np.zeros(N)
    dm = np.zeros(N)
    dc = np.zeros(N)

    Wu = np.hstack((dxx, dyy, dzz, dP, drho, dvx, dvy, dvz, de, dm, dc))

    return Wu

gamma = 1.4

mat = inn[:, 0]
maty = inn[:, 1]
matz = inn[:, 2]
matv = inn[:, 3]
matvy = inn[:, 4]
matvz = inn[:, 5]
matm = inn[:, 6]
matrho = inn[:, 7]
matp = inn[:, 8]
mate = matp / ((gamma-1)*matrho)
matc = np.zeros(len(inn))


# Initial values for integrator
s0  = np.hstack((mat, maty, matz, matp, matrho, matv, matvy, matvz, mate, matm, matc))

# Time evaluation
time = np.arange(0, steps*time_step, time_step)

# Solve integration
s    = solve_ivp(Derivatives, [0,steps*time_step], s0, t_eval=time)

# Properties obtained from integration
t       = s.t
posx    = s.y[:N]
posy    = s.y[N:2*N]
posz    = s.y[2*N:3*N]
pres    = s.y[3*N:4*N]
dens    = s.y[4*N:5*N]
velx    = s.y[5*N:6*N]
vely    = s.y[6*N:7*N]
velz    = s.y[7*N:8*N]
en      = s.y[8*N:9*N]
mas     = s.y[9*N*N:10*N]
gamma   = 1.4
P       = (gamma-1)*dens*en

print(posx.shape, dens.shape)

def updateplotting(pres, velo, densi, energ, yp):
    '''This function plots the various quantities in an updating manner. Set the input parameters
    to True if you want that specific plot.'''
    
    if pres:
        for i in range(steps):
            if i % 10 == 0:
                plt.xlabel('Position [m]')
                plt.ylabel('Pressure [W/m2]')
                plt.scatter(posx[:, i], P[:, i], label = 'Time = {} ms'.format(i))
                plt.legend()
                plt.figure()

    if energ:
        for i in range(steps):
            if i % 10 == 0:
                plt.xlabel('Position [m]')
                plt.ylabel('Internal Energy [J/kg]')
                plt.scatter(posx[:, i], en[:, i], label = 'Time = {} ms'.format(i))
                plt.legend()
                plt.figure()
            
    if velo:
        for i in range(steps):
            if i % 10 == 0:
                plt.xlabel('Position [m]')
                plt.ylabel('Velocity [m/s]')
                plt.scatter(posx[:, i], velx[:, i], label = 'Time = {} ms'.format(i))
                plt.legend()
                plt.figure()
        
    if densi:
        for i in range(steps):
            if i % 10 == 0:
                plt.xlabel('Position [m]')
                plt.ylabel('Density [W/m2]')
                plt.scatter(posx[:, i], dens[:, i], label = 'Time = {} ms'.format(i))
                plt.legend()
                plt.figure()
                
    if yp:
        for i in range(steps):
            if i % 10 == 0:
                fig = plt.figure(figsize = (12,12))
                ax = fig.add_subplot(111, projection='3d')
                p = ax.scatter(posx[:, i],posy[:, i],posz[:, i], c = dens[:, i],cmap='viridis')
                fig.colorbar(p, label = r'Density [kg/m$^3$]')
                fig = plt.figure()
    return 
print(updateplotting(pres = False, velo = False, densi = False , energ = False, yp = False))

def subplotting(pres, velo, densi, energ):
    '''This function plots the various quantities.'''
    for i in range(steps):
        if i ==100: # can use i % x == 0 to update subplots, where x is some number in the interval [0, steps]
            #fig = plt.figure(figsize = (16,12))
            #plt.suptitle('Tube properties at {} ms'.format(5*i))
            #plt.subplot(221)
            #plt.ylim(0,1.2)
            #plt.xlabel('Position [m]')
            #plt.ylabel(r'Pressure [N/m$^2$]')
            #plt.xlim(-0.6,0.6)
            #plt.scatter(posx[:, i], P[:, i], label = 'Time = {} ms'.format(i*5))
            
            #plt.subplot(222)
            #plt.ylim(0,3)
            #plt.xlabel('Position [m]')
            #plt.ylabel('Internal Energy [J/kg]')
            #plt.xlim(-0.6,0.6)
            #plt.scatter(posx[:, i], en[:, i], color = 'green', label = 'Time = {} ms'.format(i*5))
            
            #plt.subplot(223)
            #plt.ylim(-2,2)
            #plt.xlabel('Position [m]')
            #plt.ylabel('Velocity [m/s]')
            #plt.xlim(-0.6,0.6)
            #plt.scatter(posx[:, i], velx[:, i], color = 'yellow',label = 'Time = {} ms'.format(i*5))
            
            #plt.subplot(224)
            #plt.text(-0.3, 1.05, 'Rare fraction wave', fontsize = 12)
            #plt.text(0,0.62, 'Contact discontinuity', fontsize = 12)
            #plt.text(0.34, 0.32, 'Shock wave', fontsize = 12)
            #plt.ylim(0,1.2)
            #plt.xlabel('Position [m]')
            #plt.ylabel(r'Density [kg/m$^3$]')
            #plt.xlim(-0.6,0.6)
            #plt.scatter(posx[:, i], dens[:, i], color = 'red',label = 'Time = {} ms'.format(i*5))
            #plt.savefig('tube_at_40ms_w_labels.png', dpi = 300, bbox_inches = 'tight')
            fig = plt.figure(figsize = (12,12))
            ax = fig.add_subplot(111, projection='3d')
            p = ax.scatter(posx[:, i],posy[:, i],posz[:, i], c = dens,cmap='viridis')
            fig.colorbar(p, label = r'Density [kg/m$^3$]')
            plt.figure()


    return 'Here are some nice plots and animation :)'
#print(subplotting(pres = True, velo = True, densi = True, energ = True))


'''ANIMTATION'''
import numpy as np
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.animation
plt.style.use('dark_background')

AA = len(statevec)

fig = plt.figure(figsize=(18, 8)) #Determines size of figure
grids = fig.add_gridspec(2, 4)
ax5 = fig.add_subplot(grids[0:2, 2:4], projection='3d')
ax5.set_xlim(-2e8, 2e8)
ax5.set_ylim(-2e8, 2e8)
ax5.set_zlim(-2e8, 2e8)
ax5.set_xlabel('\n' + 'x [m]', linespacing = 2)
ax5.set_ylabel('\n' + 'y [m]', linespacing = 2)
ax5.set_zlabel('\n' + 'z [m]', linespacing = 2)
print('ola',posx.shape)
graph = ax5.scatter(posx[:,0],posy[:,0],posz[:,0], c = dens[:,0], cmap = 'viridis')
cb = plt.colorbar(graph, shrink = 0.7, pad = 0.1, label = r'$\rho$ [kg/m$^3$]')

ax1 = fig.add_subplot(grids[0, 0])
ax1.set_xlim(-2e8, 2e8)
ax1.set_ylabel(r'Energy [J/kg]')
ax1.set_xlabel('Position [m]')
ax1.legend()

ax2 = fig.add_subplot(grids[0, 1])
ax2.set_xlim(-2e8, 2e8)
ax2.set_ylabel(r'Velocity [m/s]')
ax2.set_xlabel('Position [m]')
ax2.legend()

ax3 = fig.add_subplot(grids[1, 0])
ax3.set_xlim(-2e8, 2e8)
ax3.set_ylabel(r'Density [kg/m$^3$]')
ax3.set_xlabel('Position [m]')
ax3.legend()

ax4 = fig.add_subplot(grids[1, 1])
ax4.set_xlim(-2e8, 2e8)
ax4.set_ylabel(r'Pressure [N/m$^2$]')
ax4.set_xlabel('Position [m]')
ax4.legend()

lineP, = ax4.plot([], [], 'bo', label = 'Planet 2')
lineP2, = ax4.plot([], [], 'o', color = 'pink',  label = 'Planet 1')
ax4.legend(fontsize = 10)
linerho, = ax3.plot([], [], 'ro', label = r'Planet 2' )
linerho2, = ax3.plot([], [], 'o',color = 'orange', label = r'Planet 1' )
ax3.legend(fontsize = 10)
lineen, = ax1.plot([], [], 'go' , label = r'Planet 2')
lineen2, = ax1.plot([], [], 'o' ,color = 'purple',  label = r'Planet 1')
ax1.legend(fontsize = 10)
linevel, = ax2.plot([], [], 'yo', label = r'Planet 2')
linevel2, = ax2.plot([], [], 'o',color = 'white', label = r'Planet 1')
ax2.legend(fontsize = 10)

time_text = ax2.text(1, 1, '', transform=ax2.transAxes)

def animate(i):
    tid = time_step*i

    graph._offsets3d = (posx[:, i], posy[:, i], posz[:, i])
    graph.set_array(dens[:, i])
    graph.set_clim(np.min(dens[:, i]), np.max(dens[:, i]))
    time_text.set_text('Time = %.1f s' % tid)
    cb.solids.set(alpha=1)
    
    
    ax1.set_ylim(np.min(en[:, i]), np.max(en[:, i]))
    
    ax2.set_ylim(np.min(velx[:, i] - 5e5), np.max(velx[:, i] + 5e5))
    ax3.set_ylim(np.min(dens[:, i]), np.max(dens[:, i]))
    ax4.set_ylim(np.min(P[:, i]), np.max(P[:, i]))
    
    
    xP = posx[:, i]
    yP = P[:, i]
    lineP.set_data(posx[:AA, i], P[:AA, i])
    lineP2.set_data(posx[AA:, i], P[AA:, i])
    
    yrho = dens[:, i]
    linerho.set_data(posx[:AA, i], dens[:AA, i])
    linerho2.set_data(posx[AA:, i], dens[AA:, i])
    
    yen = en[:, i]
    lineen.set_data(posx[:AA, i], en[:AA, i])
    lineen2.set_data(posx[AA:, i], en[AA:, i])
    
    yvel = velx[:, i]
    linevel.set_data(posx[:AA, i], velx[:AA, i])
    linevel2.set_data(posx[AA:, i], velx[AA:, i])
    
    
    return linevel, linerho, lineen, linevel, graph, 

'''Animation, comment these lines if you don't want to animate'''
anim = animation.FuncAnimation(fig, animate, 
                               frames=steps, interval=60, blit=False)
plt.tight_layout()
plt.draw()
plt.close(fig)
HTML(anim.to_html5_video())
#anim.save('hitnrun_collision_low.mp4', writer='ffmpeg', fps=60)


/Users/maja/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:100: RuntimeWarning: invalid value encountered in true_divide
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.


(602, 1) (602, 1)
None
ola (602, 1)
[ 9.75876000e+07  1.02412400e+08  9.39702000e+07  9.16201000e+07
  9.37893000e+07  1.00089779e+08  1.03441100e+08  1.01180600e+08
  9.63400000e+07  9.41883000e+07  9.76511000e+07  1.04519700e+08
  1.08560100e+08  1.05429100e+08  1.01742100e+08  1.06029800e+08
  9.83357000e+07  9.60144000e+07  1.03610900e+08  1.06798000e+08
  1.01841400e+08  9.34029000e+07  8.77600000e+07  8.83970000e+07
  9.48418000e+07  1.03759800e+08  1.10993000e+08  1.13451000e+08
  1.10221000e+08  1.02692700e+08  9.38455000e+07  8.70580000e+07
  8.48800000e+07  8.81490000e+07  9.57218000e+07  1.04871400e+08
  1.12239000e+08  1.15041000e+08  1.12149000e+08  1.04655100e+08
  9.56520000e+07  8.91500000e+07  8.83780000e+07  9.40256000e+07
  1.03259500e+08  1.10437000e+08  1.10127000e+08  1.02006300e+08
  9.48104000e+07  1.00945850e+08  1.01664300e+08  1.06306400e+08
  1.00501540e+08  1.04008000e+08  1.12600000e+08  1.16164000e+08
  1.11663000e+08  1.02327800e+08  9.36037000e+07  8.97

 -8.79050000e+07 -9.72092000e+07]
(602, 1)


## Triple planet collision

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from array import *
import random
from scipy import *
from scipy.spatial.distance import pdist
import numpy as np
from matplotlib import cm, colors, pyplot as plt
from matplotlib import animation
from IPython.display import HTML
import copy
import ffmpeg
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
%matplotlib inline


statevec = np.loadtxt('Planet600.dat')
statevec2 = np.loadtxt('Planet300.dat')
statevec3 = np.loadtxt('Planet300.dat')
statevec[:, 0] = statevec[:, 0] + 1e8
statevec[:, 3] = statevec[:, 3] - 4e5

statevec2[:, 0] = statevec2[:, 0] - 1e8
statevec2[:, 3] = statevec2[:, 3] + 4e5

statevec3[:, 2] = statevec3[:, 2]  + 1e8
statevec3[:, 5] = statevec3[:, 5] - 4e5

inn1 = np.append(statevec, statevec2, axis = 0)
inn = np.append(inn1, statevec3, axis = 0)


gamma = 1.4

plt.style.use('presentation-Copy1.mplstyle')

# Properties for the integrator
time_step   = 8
steps       = 300 # Number of steps to integrate

# Global parameters
N = len(inn)
G = 6.67408e-11



def Smooth(x,h):
    kappa = 2
    H = (h + h[:,np.newaxis])/2

    alpha = 3/(2*np.pi*H**3)
    sl = kappa*H
    dx = x - x[:,np.newaxis]
    r = np.linalg.norm(dx, axis = 2)

    R = r/H
 
    # Conditions for kernel function
    R1 = (r < H) & (R >= 0) # since R = |dx|/h < 1 --> |dx| < h
    R2 = (H <= r) & (r < 2*H) # Since R = |dx|/h < 2 --> |dx| < 2*h 
    W = np.zeros((N,N))
    
    # solve equation with conditions  

    W[R1] = (2/3) - (R[R1]**2) + (1/2)*(R[R1]**3)
    W[R2] = (1/6)*(2- R[R2])**3
    val_W = alpha*W

    R = R.reshape(N,N,1)
    r = r.reshape(N,N,1)  
    H = H.reshape(N,N,1)
    alpha = alpha.reshape(N,N,1)
    
    dW = np.zeros((N,N,3))
    dW[R1] = (-2 + (3/2)*R[R1]) * (dx[R1])/(H[R1]**2)
    dW[R2] = - (1/2)*((2 - R[R2])**2) * (dx[R2])/(H[R2]*r[R2])
    dW[R2] = np.nan_to_num(dW[R2])
 
    val_dW = alpha*dW

    return val_W, val_dW, H

def gravpotder(x, h):
    H = (h + h[:,np.newaxis])/2
    dx = x - x[:,np.newaxis]
    r = np.linalg.norm(dx, axis = 2)
    
    R = r/H
    
    unitv = dx/(r.reshape(N,N,1))
    unitv = np.nan_to_num(unitv)
    
    dF = np.zeros((N,N))

    dC1 = (0 <= R) & (R <= 1)
    dC2 = (1 <= R) & (R <= 2)
    dC3 = (R >= 2)
    
    dF[dC1] = (1/(H[dC1]**2))*((4/3)*R[dC1] - (6/5)*R[dC1]**3 + 0.5*R[dC1]**4)
    dF[dC2] = (1/(H[dC2]**2))*((8/3)*R[dC2] - 3*R[dC2]**2 + (6/5)*R[dC2]**3 - (1/6)*(R[dC2]**4) - (1/(15*R[dC2]**2)))
    dF[dC3] = 1/(r[dC3]**2)
               
    return unitv*(dF.reshape(N,N,1))

def gravpot(x, h):
    H = (h + h[:,np.newaxis])/2
    dx = x - x[:,np.newaxis]
    r = np.linalg.norm(dx, axis = 2)
    
    R = r/H
    
    F = np.zeros((N,N))
    
    C1 = (r <= H) & (R >= 0)
    C2 = (H <= r) & (r <= 2*H)
    C3 = (R >= 2)

    F[C1] = (1/H[C1]) * ((2/3)*R[C1]**2 - (10/3)*R[C1]**3 + (1/10)*R[C1]**5 - (7/5))
    F[C2] = (1/H[C2]) * ((4/3)*R[C2]**2 - R[C2]**3 + (3/10)*R[C2]**4 - (1/30)*R[C2]**5 - (8/5) + (1)/(15*R[C2]))
    F[C3] = -1/r[C3]
        
    return F

    
    
def Derivatives(t, r, boundary = False):
    '''This function calculates the evolution of the properties: Density, pressure, position, internal energy
    and velocity for the particles. The state vector r is evolved using an inbuilt integrator the solves
    the coupled ODEs, given as the co-moving derivatives in the Navier stokes equations.
    If you want to include no-slip boundary condition, set boundary = True'''
    
    # Constants
    gamma = 1.4
    eta = 1.2

    # What is to be solved: State vector
    x = np.array(r[:N])
    y = np.array(r[N:2*N])
    z = np.array(r[2*N:3*N])
    P = np.array(r[3*N:4*N])
    rho = np.array(r[4*N:5*N])
    v = np.array(r[5*N:6*N])
    vy = np.array(r[6*N:7*N])
    vz = np.array(r[7*N:8*N])
    e = np.array(r[8*N:9*N])
    m = r[9*N:10*N]
    c = np.array(r[10*N:11*N])
    
    posi = np.zeros((N,3))
    posi[:, 0] = x
    posi[:, 1] = y
    posi[:, 2] = z
    
    veli = np.zeros((N,3))
    veli[:, 0] = v
    veli[:, 1] = vy
    veli[:, 2] = vz

    h = np.ones((len(rho),))*1.6e7 # Use for more stable solution
    #h = eta*(m/rho)**(1/3)
    hij = (1/2)*(h + h[:, np.newaxis])
    hij = hij.reshape(N,N)
    
    # Kernel function values
    W, dW, DX = Smooth(posi, h) # dx = velocity derivative
    
    #Smoothing kernel values
    F = gravpot(posi, h)
    dF = gravpotder(posi, h)
    
    
    # Density
    sum_dens = m[:, np.newaxis]*W
    rho = np.sum(sum_dens, axis = 1)
    
    # Pressure 
    P = (gamma-1)*rho*e

    # Update stuff
    r[3*N:4*N] = P
    r[4*N:5*N] = rho
   
    # Viscosity equations
    phiij = 0.1*hij
    phiij = phiij.reshape(N,N)
    
    # Speed of sound
    c = np.sqrt((gamma-1)*e)
    c = np.nan_to_num(c)
    cij= 0.5*(c + c[:,np.newaxis])
    
    # Distance and velocity
    vij = veli - veli[:, np.newaxis]
    xij = posi - posi[:, np.newaxis]

    vxij = np.sum(vij*xij, axis = 2)
    xxij = np.sum(xij**2, axis = 2)
    
    # Change of density
    rhoij = 0.5*(rho + rho[:, np.newaxis])
    #cond = vij*xij
    cond = vxij
    
    # Conditions for largepi 
    cond1 = (cond < 0)
    cond2 = (cond >= 0)
    PIij = np.zeros(cond1.shape)

    varphiij = (hij*vxij)/((xxij) + phiij**2)
    
    PIij[cond1] = (-cij[cond1]*varphiij[cond1] + varphiij[cond1]**2)/(rhoij[cond1])
    PIij[cond2] = 0

    largepi = PIij

    # i and j values of pressure and density
    PI = P
    PJ = P[:, np.newaxis]
    rhoi = rho
    rhoj = rho[:, np.newaxis]
    pressure_and_density_fraction = ((PI/(rhoi**2)) + (PJ/(rhoj**2)) + largepi)*m[:, np.newaxis]
    

    # Acceleration gravity
    Gi = gravpotder(posi, h)
    Gj = gravpotder(posi, h.T)
    gdvdt = - (G/2)*np.sum(m[:, np.newaxis]*(Gi + Gj), axis = 1)

    
    dvx_calc = pressure_and_density_fraction*dW[:, :, 0]
    dvx = -np.sum(dvx_calc, axis=0) + gdvdt[:, 0]
    
    dvy_calc = pressure_and_density_fraction*dW[:, :, 1]
    dvy = -np.sum(dvy_calc, axis=0) + gdvdt[:, 1]
    
    dvz_calc = pressure_and_density_fraction*dW[:, :, 2]
    dvz = -np.sum(dvz_calc, axis=0) + gdvdt[:, 2]

    
    
    # Dirichlet no-slip boundary condition
    if boundary:
        Z1 = (-0.6*0.95 >= x)                     
        Z2 = (x >= 0.6*0.95)                    

        dv[Z1] = 0            
        dv[Z2] = 0
        
    # Energy 
    ss = (m[:, np.newaxis])*(PI/(rhoi**2) + PJ/(rhoj**2) + largepi)
    #print(ss.shape, 'ss', 'vij', vij.shape, 'dW', dW.shape)
    
    de = np.einsum('ij,ijk->ijk', ss, vij)
    de = np.einsum('ijk, ijk->ij', de, dW)
    de = np.einsum('ij->i', de)
    de = 0.5*de

    # velocity
    dxx = veli[:, 0]
    dyy = veli[:, 1]
    dzz = veli[:, 2]
    
    # Derivatives 
    drho = np.zeros(N)
    dP = np.zeros(N)
    dm = np.zeros(N)
    dc = np.zeros(N)

    Wu = np.hstack((dxx, dyy, dzz, dP, drho, dvx, dvy, dvz, de, dm, dc))

    return Wu

gamma = 1.4

mat = inn[:, 0]
maty = inn[:, 1]
matz = inn[:, 2]
matv = inn[:, 3]
matvy = inn[:, 4]
matvz = inn[:, 5]
matm = inn[:, 6]
matrho = inn[:, 7]
matp = inn[:, 8]
mate = matp / ((gamma-1)*matrho)
matc = np.zeros(len(inn))


# Initial values for integrator
s0  = np.hstack((mat, maty, matz, matp, matrho, matv, matvy, matvz, mate, matm, matc))

# Time evaluation
time = np.arange(0, steps*time_step, time_step)

# Solve integration
s    = solve_ivp(Derivatives, [0,steps*time_step], s0, t_eval=time)


# Properties obtained from integration
t       = s.t
posx    = s.y[:N]
posy    = s.y[N:2*N]
posz    = s.y[2*N:3*N]
pres    = s.y[3*N:4*N]
dens    = s.y[4*N:5*N]
velx    = s.y[5*N:6*N]
vely    = s.y[6*N:7*N]
velz    = s.y[7*N:8*N]
en      = s.y[8*N:9*N]
mas     = s.y[9*N*N:10*N]
gamma   = 1.4
P       = (gamma-1)*dens*en

print(posx.shape, dens.shape)





def updateplotting(pres, velo, densi, energ, yp):
    '''This function plots the various quantities and an updating manner. Set the input parameters
    to True if wanting the plot.'''
    
    if pres:
        for i in range(steps):
            if i % 10 == 0:
                plt.xlabel('Position [m]')
                plt.ylabel('Pressure [W/m2]')
                plt.scatter(posx[:, i], P[:, i], label = 'Time = {} ms'.format(i))
                plt.legend()
                plt.figure()

    if energ:
        for i in range(steps):
            if i % 10 == 0:
                plt.xlabel('Position [m]')
                plt.ylabel('Internal Energy [J/kg]')
                plt.scatter(posx[:, i], en[:, i], label = 'Time = {} ms'.format(i))
                plt.legend()
                plt.figure()
            
    if velo:
        for i in range(steps):
            if i % 10 == 0:
                plt.xlabel('Position [m]')
                plt.ylabel('Velocity [m/s]')
                plt.scatter(posx[:, i], velx[:, i], label = 'Time = {} ms'.format(i))
                plt.legend()
                plt.figure()
        
    if densi:
        for i in range(steps):
            if i % 10 == 0:
                plt.xlabel('Position [m]')
                plt.ylabel('Density [W/m2]')
                plt.scatter(posx[:, i], dens[:, i], label = 'Time = {} ms'.format(i))
                plt.legend()
                plt.figure()
                
    if yp:
        for i in range(steps):
            if i % 10 == 0:
                fig = plt.figure(figsize = (12,12))
                ax = fig.add_subplot(111, projection='3d')
                p = ax.scatter(posx[:, i],posy[:, i],posz[:, i], c = dens[:, i],cmap='viridis')
                fig.colorbar(p, label = r'Density [kg/m$^3$]')
                fig = plt.figure()
    return 
print(updateplotting(pres = False, velo = False, densi = False , energ = False, yp = False))

def subplotting(pres, velo, densi, energ):
    '''This function plots the various quantities.'''
    for i in range(steps):
        if i ==100: # can use i % x == 0 to update subplots, where x is some number in the interval [0, steps]
            #fig = plt.figure(figsize = (16,12))
            #plt.suptitle('Tube properties at {} ms'.format(5*i))
            #plt.subplot(221)
            #plt.ylim(0,1.2)
            #plt.xlabel('Position [m]')
            #plt.ylabel(r'Pressure [N/m$^2$]')
            #plt.xlim(-0.6,0.6)
            #plt.scatter(posx[:, i], P[:, i], label = 'Time = {} ms'.format(i*5))
            
            #plt.subplot(222)
            #plt.ylim(0,3)
            #plt.xlabel('Position [m]')
            #plt.ylabel('Internal Energy [J/kg]')
            #plt.xlim(-0.6,0.6)
            #plt.scatter(posx[:, i], en[:, i], color = 'green', label = 'Time = {} ms'.format(i*5))
            
            #plt.subplot(223)
            #plt.ylim(-2,2)
            #plt.xlabel('Position [m]')
            #plt.ylabel('Velocity [m/s]')
            #plt.xlim(-0.6,0.6)
            #plt.scatter(posx[:, i], velx[:, i], color = 'yellow',label = 'Time = {} ms'.format(i*5))
            
            #plt.subplot(224)
            #plt.text(-0.3, 1.05, 'Rare fraction wave', fontsize = 12)
            #plt.text(0,0.62, 'Contact discontinuity', fontsize = 12)
            #plt.text(0.34, 0.32, 'Shock wave', fontsize = 12)
            #plt.ylim(0,1.2)
            #plt.xlabel('Position [m]')
            #plt.ylabel(r'Density [kg/m$^3$]')
            #plt.xlim(-0.6,0.6)
            #plt.scatter(posx[:, i], dens[:, i], color = 'red',label = 'Time = {} ms'.format(i*5))
            #plt.savefig('tube_at_40ms_w_labels.png', dpi = 300, bbox_inches = 'tight')
            fig = plt.figure(figsize = (12,12))
            ax = fig.add_subplot(111, projection='3d')
            p = ax.scatter(posx[:, i],posy[:, i],posz[:, i], c = dens,cmap='viridis')
            fig.colorbar(p, label = r'Density [kg/m$^3$]')
            plt.figure()


    return 'Here are some nice plots and animation :)'
#print(subplotting(pres = True, velo = True, densi = True, energ = True))


'''ANIMTATION'''
import numpy as np
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.animation
plt.style.use('dark_background')

AA = len(statevec)
BB = 900

fig = plt.figure(figsize=(18, 8)) #Determines size of figure
grids = fig.add_gridspec(2, 4)
ax5 = fig.add_subplot(grids[0:2, 2:4], projection='3d')
ax5.set_xlim(-2e8, 2e8)
ax5.set_ylim(-2e8, 2e8)
ax5.set_zlim(-2e8, 2e8)
ax5.set_xlabel('\n' + 'x [m]', linespacing = 2)
ax5.set_ylabel('\n' + 'y [m]', linespacing = 2)
ax5.set_zlabel('\n' + 'z [m]', linespacing = 2)

graph = ax5.scatter(posx[:,0],posy[:,0],posz[:,0], c = dens[:,0], cmap = 'viridis')
cb = plt.colorbar(graph, shrink = 0.7, pad = 0.1, label = r'$\rho$ [kg/m$^3$]')

ax1 = fig.add_subplot(grids[0, 0])
ax1.set_xlim(-2e8, 2e8)
ax1.set_ylabel(r'Energy [J/kg]')
ax1.set_xlabel('Position [m]')


ax2 = fig.add_subplot(grids[0, 1])
ax2.set_xlim(-2e8, 2e8)
ax2.set_ylabel(r'Velocity [m/s]')
ax2.set_xlabel('Position [m]')


ax3 = fig.add_subplot(grids[1, 0])
ax3.set_xlim(-2e8, 2e8)
ax3.set_ylabel(r'Density [kg/m$^3$]')
ax3.set_xlabel('Position [m]')


ax4 = fig.add_subplot(grids[1, 1])
ax4.set_xlim(-2e8, 2e8)
ax4.set_ylabel(r'Pressure [N/m$^2$]')
ax4.set_xlabel('Position [m]')

lineP, = ax4.plot([], [], 'bo', label = 'Planet 2')
lineP2, = ax4.plot([], [], 'o', color = 'pink',  label = 'Planet 1')
lineP3, = ax4.plot([], [], 'o', color = 'purple',  label = 'Planet 3')
ax4.legend(fontsize = 10)

linerho, = ax3.plot([], [], 'ro', label = 'Planet 2' )
linerho2, = ax3.plot([], [], 'o',color = 'orange', label = r'Planet 1' )
linerho3, = ax3.plot([], [], 'o',color = 'green', label = r'PLanet 3' )
ax3.legend(fontsize = 10)

lineen, = ax1.plot([], [], 'go' , label = r'Planet 2')
lineen2, = ax1.plot([], [], 'o' ,color = 'purple',  label = r'Planet 1')
lineen3, =  ax1.plot([], [], 'o' ,color = 'red',  label = r'Planet 3')
ax1.legend(fontsize = 10)

linevel, = ax2.plot([], [], 'yo', label = r'Planet 2')
linevel2, = ax2.plot([], [], 'o', color = 'white', label = r'PLanet 1')
linevel3, = ax2.plot([], [], 'o', color = 'cyan', label = r'PLanet 3')
ax2.legend(fontsize = 10)

time_text = ax2.text(1, 1, '', transform=ax2.transAxes)


def animate(i):
    tid = time_step*i

    graph._offsets3d = (posx[:, i], posy[:, i], posz[:, i])
    graph.set_array(dens[:, i])
    graph.set_clim(np.min(dens[:, i]), np.max(dens[:, i]))
    time_text.set_text('Time = %.1f s' % tid)
    cb.solids.set(alpha=1)
    
    
    ax1.set_ylim(np.min(en[:, i]), np.max(en[:, i]))
    
    ax2.set_ylim(np.min(velx[:, i] - 5e5), np.max(velx[:, i] + 5e5))
    ax3.set_ylim(np.min(dens[:, i]), np.max(dens[:, i]))
    ax4.set_ylim(np.min(P[:, i]), np.max(P[:, i]))
    
    
    xP = posx[:, i]
    yP = P[:, i]
    lineP.set_data(posx[:AA, i], P[:AA, i])
    lineP2.set_data(posx[AA:BB, i], P[AA:BB, i])
    lineP3.set_data(posx[BB:, i], P[BB:, i])
    
    yrho = dens[:, i]
    linerho.set_data(posx[:AA, i], dens[:AA, i])
    linerho2.set_data(posx[AA:BB, i], dens[AA:BB, i])
    linerho3.set_data(posx[BB:, i], dens[BB:, i])
    
    yen = en[:, i]
    lineen.set_data(posx[:AA, i], en[:AA, i])
    lineen2.set_data(posx[AA:BB, i], en[AA:BB, i])
    lineen3.set_data(posx[BB:, i], en[BB:, i])
    
 
    yvel = velx[:, i]
    linevel.set_data(posx[:AA, i], velx[:AA, i])
    linevel2.set_data(posx[AA:900, i], velx[AA:BB, i])
    linevel3.set_data(posx[900:, i], velx[BB:, i])
    
    
    return linevel, linerho, lineen, linevel, graph, 

'''Animation, comment these lines if you don't want to animate'''
anim = animation.FuncAnimation(fig, animate, 
                               frames=steps, interval=60, blit=False)
plt.tight_layout()
plt.draw()
plt.close(fig)
HTML(anim.to_html5_video())
#anim.save('head_on_collision_3planets.mp4', writer='ffmpeg', fps=30)


/Users/maja/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:93: RuntimeWarning: invalid value encountered in true_divide


(1201, 300) (1201, 300)
None


In [23]:
A = np.array([1,2,3])
B = np.array([1,2,3])
s = np.vstack((A,B))

[[1 2 3]
 [1 2 3]]
[2. 2.]
